In [70]:
from typing import Tuple

with open("inputs/day14-input.txt") as f:
    puzzle = f.read().splitlines()

    
def parse_chemical(chemical: str) -> Tuple[int, str]:
    chemical_quantity, chemical_name = chemical.split()
    return (int(chemical_quantity), chemical_name)


reactions = {}
for line in puzzle:
    input_chemicals, output_chemical = line.split(" => ")
    
    output_chemical_quantity, output_chemical_name = parse_chemical(output_chemical)
    
    reactions[output_chemical_name] = {
        "quantity": output_chemical_quantity,
        "inputs": list(map(parse_chemical, input_chemicals.split(", ")))
    }

## Part 1

In [51]:
%%time

import math
from typing import Dict

def production_finished(production: Dict[str, int]) -> bool:
    quantities = [production[x] for x in production if x != "ORE"]
    
    return all(map(lambda x: x <= 0, quantities))

def produce_fuel(quantity: int) -> int:
    production = {"FUEL": quantity}
    
    while not production_finished(production):
        chemical = next(filter(lambda x: x != "ORE" and production[x] > 0, production))

        required_quantity = production[chemical]
        reaction_quantity = reactions[chemical]["quantity"]
        production_factor = math.ceil(required_quantity / reaction_quantity)

        for input_quantity, input_name in reactions[chemical]["inputs"]:
            production[input_name] = production.get(input_name, 0) + production_factor * input_quantity
        
        production[chemical] -= production_factor * reaction_quantity
        
    return production["ORE"]
    
print(produce_fuel(1))

612880
CPU times: user 5.43 ms, sys: 0 ns, total: 5.43 ms
Wall time: 5.41 ms


## Part 2

In [102]:
%%time

cargo_hold = 1000000000000
lower_bound = 0
upper_bound = 1

while produce_fuel(upper_bound) < cargo_hold:
    lower_bound = upper_bound
    upper_bound *= 2
    
while lower_bound + 1 != upper_bound:
    middle = (lower_bound + upper_bound) // 2
    
    if produce_fuel(middle) > cargo_hold:
        upper_bound = middle
    else:
        lower_bound = middle
        
print(lower_bound)

2509120
CPU times: user 119 ms, sys: 900 µs, total: 120 ms
Wall time: 121 ms
